<a href="https://colab.research.google.com/github/Anderson-Andre-P/Summary-Generator-With-Python-and-NLTK/blob/main/pdf_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 12.4 MB/s eta 0:00:00


In [28]:
import PyPDF2

from google.colab import files

In [6]:
uploaded = files.upload()

Saving Resenha Artigo - Anderson André.pdf to Resenha Artigo - Anderson André.pdf


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [58]:
pdf_file = open('/content/Resenha Artigo - Anderson André.pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)

In [59]:
pdf_text = ""

In [60]:
from PyPDF2 import PdfReader


In [61]:
reader = PdfReader('/content/Resenha Artigo - Anderson André.pdf')
for page in reader.pages:
    pdf_text += page.extract_text()
    # print(page.extract_text())

In [62]:
print(f"pdf page count : {len(reader.pages)}")

pdf page count : 3


In [63]:
print(pdf_text)

Nome:
Anderson
André
Pereira
Eleutério;
RA:
015642.
1.
Introdução
Segundo
Fadel
e
Silveira
(2010),
as
metodologias
ágeis
são
muito
utilizadas
por
empresas
desenvolvedoras
de
software
que
visam
administrar
melhor
seus
recursos
e
o
tempo
disponível
para
desenvolvimento
das
atividades.
Um
dos
objetivos
que
as
organizações
têm
ao
utilizar
as
metodologias
ágeis
é
melhorar
o
fluxo
do
desenvolvimento
de
software,
alocando
menos
recursos
na
produção
e
aumentando
a
eficiência
das
equipes,
isso
melhora
a
qualidade
dos
produtos
desenvolvidos.
Os
mesmos
autores,
informam
que
o
um
encontro
realizado
por
17
desenvolvedores
de
software
identificou
12
princípios
e
4
premissas
que
estão
presentes
no
manifesto
ágil.
As
4
premissas
podem
ser
representados
nos
seguintes
tópicos:
●
Valorizar
pessoas
e
interações
acima
de
processos
e
ferramentas;
●
Priorizar
o
software
funcional
em
vez
de
documentação
detalhada;
●
Dar
importância
à
colaboração
com
o
cliente
em
vez
de
negociação
de
contratos;
●
Ser
adaptável

In [64]:
pdf_text = ' '.join(pdf_text.split())

In [65]:
print(pdf_text)

Nome: Anderson André Pereira Eleutério; RA: 015642. 1. Introdução Segundo Fadel e Silveira (2010), as metodologias ágeis são muito utilizadas por empresas desenvolvedoras de software que visam administrar melhor seus recursos e o tempo disponível para desenvolvimento das atividades. Um dos objetivos que as organizações têm ao utilizar as metodologias ágeis é melhorar o fluxo do desenvolvimento de software, alocando menos recursos na produção e aumentando a eficiência das equipes, isso melhora a qualidade dos produtos desenvolvidos. Os mesmos autores, informam que o um encontro realizado por 17 desenvolvedores de software identificou 12 princípios e 4 premissas que estão presentes no manifesto ágil. As 4 premissas podem ser representados nos seguintes tópicos: ● Valorizar pessoas e interações acima de processos e ferramentas; ● Priorizar o software funcional em vez de documentação detalhada; ● Dar importância à colaboração com o cliente em vez de negociação de contratos; ● Ser adaptável

In [66]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import heapq
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [67]:
# Tokenizando o texto em sentenças e palavras
sentencas = sent_tokenize(pdf_text)
palavras = word_tokenize(pdf_text.lower())

In [68]:
print(palavras)

['nome', ':', 'anderson', 'andré', 'pereira', 'eleutério', ';', 'ra', ':', '015642', '.', '1.', 'introdução', 'segundo', 'fadel', 'e', 'silveira', '(', '2010', ')', ',', 'as', 'metodologias', 'ágeis', 'são', 'muito', 'utilizadas', 'por', 'empresas', 'desenvolvedoras', 'de', 'software', 'que', 'visam', 'administrar', 'melhor', 'seus', 'recursos', 'e', 'o', 'tempo', 'disponível', 'para', 'desenvolvimento', 'das', 'atividades', '.', 'um', 'dos', 'objetivos', 'que', 'as', 'organizações', 'têm', 'ao', 'utilizar', 'as', 'metodologias', 'ágeis', 'é', 'melhorar', 'o', 'fluxo', 'do', 'desenvolvimento', 'de', 'software', ',', 'alocando', 'menos', 'recursos', 'na', 'produção', 'e', 'aumentando', 'a', 'eficiência', 'das', 'equipes', ',', 'isso', 'melhora', 'a', 'qualidade', 'dos', 'produtos', 'desenvolvidos', '.', 'os', 'mesmos', 'autores', ',', 'informam', 'que', 'o', 'um', 'encontro', 'realizado', 'por', '17', 'desenvolvedores', 'de', 'software', 'identificou', '12', 'princípios', 'e', '4', 'pre

In [69]:
# Removendo as stopwords e pontuações
stop_words = set(stopwords.words('portuguese'))
palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in stop_words and palavra.isalpha()]

In [70]:
print(palavras_sem_stopwords)

['nome', 'anderson', 'andré', 'pereira', 'eleutério', 'ra', 'introdução', 'segundo', 'fadel', 'silveira', 'metodologias', 'ágeis', 'utilizadas', 'empresas', 'desenvolvedoras', 'software', 'visam', 'administrar', 'melhor', 'recursos', 'tempo', 'disponível', 'desenvolvimento', 'atividades', 'objetivos', 'organizações', 'têm', 'utilizar', 'metodologias', 'ágeis', 'melhorar', 'fluxo', 'desenvolvimento', 'software', 'alocando', 'menos', 'recursos', 'produção', 'aumentando', 'eficiência', 'equipes', 'melhora', 'qualidade', 'produtos', 'desenvolvidos', 'mesmos', 'autores', 'informam', 'encontro', 'realizado', 'desenvolvedores', 'software', 'identificou', 'princípios', 'premissas', 'presentes', 'manifesto', 'ágil', 'premissas', 'podem', 'representados', 'seguintes', 'tópicos', 'valorizar', 'pessoas', 'interações', 'acima', 'processos', 'ferramentas', 'priorizar', 'software', 'funcional', 'vez', 'documentação', 'detalhada', 'dar', 'importância', 'colaboração', 'cliente', 'vez', 'negociação', 'c

In [71]:
# Aplicando o stemmer para reduzir as palavras para sua raiz
ps = PorterStemmer()
palavras_stemmed = [ps.stem(palavra) for palavra in palavras_sem_stopwords]

In [72]:
print(palavras_stemmed)

['nome', 'anderson', 'andré', 'pereira', 'eleutério', 'ra', 'introdução', 'segundo', 'fadel', 'silveira', 'metodologia', 'ágei', 'utilizada', 'empresa', 'desenvolvedora', 'softwar', 'visam', 'administrar', 'melhor', 'recurso', 'tempo', 'disponível', 'desenvolvimento', 'atividad', 'objetivo', 'organizaçõ', 'têm', 'utilizar', 'metodologia', 'ágei', 'melhorar', 'fluxo', 'desenvolvimento', 'softwar', 'alocando', 'meno', 'recurso', 'produção', 'aumentando', 'eficiência', 'equip', 'melhora', 'qualidad', 'produto', 'desenvolvido', 'mesmo', 'autor', 'informam', 'encontro', 'realizado', 'desenvolvedor', 'softwar', 'identific', 'princípio', 'premissa', 'present', 'manifesto', 'ágil', 'premissa', 'podem', 'representado', 'seguint', 'tópico', 'valorizar', 'pessoa', 'interaçõ', 'acima', 'processo', 'ferramenta', 'priorizar', 'softwar', 'funcion', 'vez', 'documentação', 'detalhada', 'dar', 'importância', 'colaboração', 'client', 'vez', 'negociação', 'contrato', 'adaptável', 'mudança', 'vez', 'seguir

In [73]:
# Criando um dicionário de frequência de palavras
frequencia_palavras = nltk.FreqDist(palavras_stemmed)

In [74]:
print(frequencia_palavras)

<FreqDist with 256 samples and 388 outcomes>


In [75]:
# Calculando a pontuação de cada sentença com base nas frequências das palavras
pontuacao_sentencas = {}
for sentenca in sentencas:
    for palavra in word_tokenize(sentenca.lower()):
        if palavra in palavras_sem_stopwords:
            if sentenca not in pontuacao_sentencas:
                pontuacao_sentencas[sentenca] = frequencia_palavras[ps.stem(palavra)]
            else:
                pontuacao_sentencas[sentenca] += frequencia_palavras[ps.stem(palavra)]

In [76]:
print(pontuacao_sentencas)

{'Nome: Anderson André Pereira Eleutério; RA: 015642.': 6, 'Introdução Segundo Fadel e Silveira (2010), as metodologias ágeis são muito utilizadas por empresas desenvolvedoras de software que visam administrar melhor seus recursos e o tempo disponível para desenvolvimento das atividades.': 53, 'Um dos objetivos que as organizações têm ao utilizar as metodologias ágeis é melhorar o fluxo do desenvolvimento de software, alocando menos recursos na produção e aumentando a eficiência das equipes, isso melhora a qualidade dos produtos desenvolvidos.': 50, 'Os mesmos autores, informam que o um encontro realizado por 17 desenvolvedores de software identificou 12 princípios e 4 premissas que estão presentes no manifesto ágil.': 29, 'As 4 premissas podem ser representados nos seguintes tópicos: ● Valorizar pessoas e interações acima de processos e ferramentas; ● Priorizar o software funcional em vez de documentação detalhada; ● Dar importância à colaboração com o cliente em vez de negociação de 

In [77]:
# Selecionando as melhores sentenças com base nas maiores pontuações
melhores_sentencas = heapq.nlargest(3, pontuacao_sentencas, key=pontuacao_sentencas.get)
resumo = ' '.join(melhores_sentencas)

In [78]:
print(melhores_sentencas)

['Scrum Fadel e Silveira (2010) diz que no Scrum, não é definida nenhuma técnica para o desenvolvimento de software durante sua implementação, na verdade, ele se concentra em descrever como cada membro da equipe deve se comportar durante o trabalho para se tornar flexível, já que o ambiente é repleto de mudanças frequentes.', 'Fadel e Silveira ( 2010) ainda diz que a metodologia ágil Scrum tem um processo que busca realizar entregas de maneira contínua, onde cada entrega é definida pelo product backlog e as entregas são realizadas em ciclos de sprint que são os prazospré determinados para entregar as atividades.', 'As 4 premissas podem ser representados nos seguintes tópicos: ● Valorizar pessoas e interações acima de processos e ferramentas; ● Priorizar o software funcional em vez de documentação detalhada; ● Dar importância à colaboração com o cliente em vez de negociação de contratos; ● Ser adaptável às mudanças em vez de seguir um plano inicial rigidamente; Os 12 princípios de acord

In [79]:
# Imprimindo o resumo gerado
print("Resumo: ", resumo)

Resumo:  Scrum Fadel e Silveira (2010) diz que no Scrum, não é definida nenhuma técnica para o desenvolvimento de software durante sua implementação, na verdade, ele se concentra em descrever como cada membro da equipe deve se comportar durante o trabalho para se tornar flexível, já que o ambiente é repleto de mudanças frequentes. Fadel e Silveira ( 2010) ainda diz que a metodologia ágil Scrum tem um processo que busca realizar entregas de maneira contínua, onde cada entrega é definida pelo product backlog e as entregas são realizadas em ciclos de sprint que são os prazospré determinados para entregar as atividades. As 4 premissas podem ser representados nos seguintes tópicos: ● Valorizar pessoas e interações acima de processos e ferramentas; ● Priorizar o software funcional em vez de documentação detalhada; ● Dar importância à colaboração com o cliente em vez de negociação de contratos; ● Ser adaptável às mudanças em vez de seguir um plano inicial rigidamente; Os 12 princípios de acor